# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [20]:
weather_data = pd.read_csv("output_data/cities.csv")
weather_data

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Kigoma,-4.88,29.63,73.45,69,33,2.37,TZ,1595462130
1,1,Carnarvon,-24.87,113.63,53.60,100,0,4.70,AU,1595462012
2,2,Touros,-5.20,-35.46,77.00,78,75,8.05,BR,1595461864
3,3,Ugoofaaru,5.67,73.00,81.75,78,98,20.62,MV,1595462131
4,4,Khatanga,71.97,102.50,47.05,81,100,16.60,RU,1595461998
...,...,...,...,...,...,...,...,...,...,...
547,547,Leshukonskoye,64.90,45.76,63.46,80,23,8.68,RU,1595462186
548,548,Flagstaff,35.20,-111.65,78.01,36,20,4.70,US,1595462049
549,549,Kashgar,39.45,75.98,68.00,40,0,8.95,CN,1595462186
550,550,São Félix do Xingu,-6.64,-51.99,76.17,51,9,2.01,BR,1595462186


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
gmaps.configure(api_key=g_key)

In [22]:

fig = gmaps.figure(map_type='SATELLITE')
heatmap_locs = weather_data[["Lat","Lng"]]
humidity= weather_data["Humidity"].astype(float)
maxintensity = humidity.max()

heatmap_layer= gmaps.heatmap_layer(
    heatmap_locs,weights=humidity,max_intensity=maxintensity, point_radius =2,dissipating=False)

fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
# A max temperature < 80 degrees but > 70.
# Wind speed < 10mph
# Cloudiness = 0
# drop any rows that don't contain any of the above


ideal_temp = weather_data.loc[
    (weather_data["Max Temp"]<80) & 
    (weather_data["Max Temp"]>70) & 
    (weather_data["Wind Speed"]<10) & 
    (weather_data["Cloudiness"]==0)]


# Dropping any null values
hotel_df = ideal_temp.dropna(how='any')
hotel_df.reset_index(inplace=True)
hotel_df




,index,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,90,90,Oktyabr'skoye,43.06,44.74,75.20,57,0,8.95,RU,1595294902
1,109,109,Mazara del Vallo,37.66,12.59,71.60,88,0,3.36,IT,1595294905
2,127,127,Tieli,46.95,128.05,75.31,63,0,9.57,CN,1595294907
3,137,137,Loralai,30.37,68.60,79.03,45,0,2.84,PK,1595294908
4,142,142,Sestri Levante,44.27,9.40,71.01,88,0,4.72,IT,1595294908
5,173,173,Marsá Maţrūḩ,31.35,27.25,71.60,88,0,5.82,EG,1595294912
6,177,177,Zhanakorgan,43.91,67.25,78.89,20,0,7.14,KZ,1595294912
7,238,238,Boende,-0.22,20.87,70.11,75,0,1.79,CD,1595294918
8,260,260,Talaya,61.38,152.75,70.11,37,0,5.97,RU,1595294922
9,286,286,Caravelas,-17.71,-39.25,70.21,89,0,6.82,BR,1595294925


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotels = []

distance=5000

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Grabbing the location based on Lat and Lng in the ideal_hotels df
for location in range(len(hotel_df)):
    latitude = hotel_df.loc[location]["Lat"]
    longitude = hotel_df.loc[location]["Lng"]

    params = {"location":f"{latitude},{longitude}",
          "radius": distance,
          "type":"hotel",
          "key":g_key}
    request = requests.get(url,params)
    request_json = request.json()
   
    try:
        hotels.append(request_json["results"][0]["name"])
    except:
        hotels.append("No hotel near")


        
hotel_df
# Adding Hotel Name column to "hotel_df"


hotel_df["Hotel Name"] = hotels

hotel_df

# print(request)    
# print(request.url)


,index,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,90,90,Oktyabr'skoye,43.06,44.74,75.20,57,0,8.95,RU,1595294902,Vladikavkaz
1,109,109,Mazara del Vallo,37.66,12.59,71.60,88,0,3.36,IT,1595294905,Mazara del Vallo
2,127,127,Tieli,46.95,128.05,75.31,63,0,9.57,CN,1595294907,Yichun
3,137,137,Loralai,30.37,68.60,79.03,45,0,2.84,PK,1595294908,Loralai
4,142,142,Sestri Levante,44.27,9.40,71.01,88,0,4.72,IT,1595294908,Sestri Levante
5,173,173,Marsá Maţrūḩ,31.35,27.25,71.60,88,0,5.82,EG,1595294912,Marsa Matruh
6,177,177,Zhanakorgan,43.91,67.25,78.89,20,0,7.14,KZ,1595294912,Zhanakorgan
7,238,238,Boende,-0.22,20.87,70.11,75,0,1.79,CD,1595294918,No hotel near
8,260,260,Talaya,61.38,152.75,70.11,37,0,5.97,RU,1595294922,No hotel near
9,286,286,Caravelas,-17.71,-39.25,70.21,89,0,6.82,BR,1595294925,Caravelas


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map

hotel_markers = gmaps.marker_layer(locations)
fig.add_layer(hotel_markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))